In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [2]:
### loading data set
data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [4]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [9]:
## convertir gender to 0 or 1

label_encoder_gender=LabelEncoder()
data["Gender"]=label_encoder_gender.fit_transform(data["Gender"])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [12]:
## Onehot encode Geography column
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder(sparse_output=False) 
geo_encoder = onehot_encoder_geo.fit_transform(data[["Geography"]])
geo_encoder

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [13]:
geo_encoder_df=pd.DataFrame(geo_encoder,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoder_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [14]:
## combine all columns for dataset original data

data=pd.concat([data.drop("Geography",axis=1),geo_encoder_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [15]:
X=data.drop(['EstimatedSalary'],axis=1)
y=data['EstimatedSalary']

In [18]:
## split the data in training and prediction
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

#scale theses features
scale=StandardScaler()
X_train=scale.fit_transform(X_train)
X_test=scale.transform(X_test)

In [19]:
with open('label_encoder_gender2.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('label_encoder_geo2.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

with open('scaler2.pkl','wb') as file:
    pickle.dump(scale,file)

In [23]:
## ANN Regression  Problem training
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [24]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),#HL1 Connected with input layer
    Dense(32,activation='relu'), # HL2
    Dense(1) ## out layer
])

## compile model
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

/Users/macbookair/miniconda3/envs/coursai/lib/python3.12/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [25]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [27]:
## set up  the tensorboard
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

log_dir="lregressionogs/fit/"+datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [28]:
## Set up Early Stoping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [29]:
## Training the model
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    callbacks=[tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 775us/step - loss: 100393.2031 - mae: 100393.2031 - val_loss: 98576.2344 - val_mae: 98576.2344
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 537us/step - loss: 99848.0859 - mae: 99848.0859 - val_loss: 97463.5391 - val_mae: 97463.5391
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 519us/step - loss: 97888.8672 - mae: 97888.8672 - val_loss: 94558.9609 - val_mae: 94558.9609
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 518us/step - loss: 93933.3047 - mae: 93933.3047 - val_loss: 89545.0547 - val_mae: 89545.0547
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 526us/step - loss: 87935.9844 - mae: 87935.9844 - val_loss: 82748.6016 - val_mae: 82748.6016
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 670us/step - loss: 80408.4375 - mae: 80408.4375 - val_loss: 74958.1094 - val_mae: 74958.1094
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 572us/step - loss: 72376.2344 - mae: 72376.2344 - val_loss: 67377.5391 - val_mae: 67377.5391
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━

In [30]:
## Evaluation du model
%load_ext tensorboard

In [40]:
%tensorboard --logdir lregressionogs/fit

Reusing TensorBoard on port 6010 (pid 77137), started 0:00:07 ago. (Use '!kill 77137' to kill it.)

In [46]:
## save model
model.save('regression_model.h5')
